## 103590450 四資四 馬茂源

In [11]:
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark
import os
import pandas as pd

In [12]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
        .setMaster("local")
        .setAppName("hw1")
        .set('spark.executor.memory', '4g'))

In [13]:
sc = SparkContext(conf=conf)
sql_sc = SQLContext(sc)

In [14]:
# rdd = (sc.textFile('./household_power_consumption.csv')
#        .map(lambda line: line.split(';')))
# header = rdd.first()
# data = sql_sc.createDataFrame(rdd.filter(lambda line: line != header), header)
# data.show()

In [15]:
# data = pd.read_csv('./household_power_consumption.csv', 
#                    delimiter=';', 
#                    dtype={'Date':str,
#                           'Time':str,
#                           'Global_active_power':str,
#                           'Global_reactive_power':str,
#                           'Voltage':str,
#                           'Global_intensity':str,
#                           'Sub_metering_1':str,
#                           'Sub_metering_2':str,
#                           'sub_metering_3':str,})
# data = sql_sc.createDataFrame(data)

In [16]:
data = sql_sc.read.csv('./household_power_consumption.csv', sep=';', header=True)

In [17]:
data.dtypes

[('Date', 'string'),
 ('Time', 'string'),
 ('Global_active_power', 'string'),
 ('Global_reactive_power', 'string'),
 ('Voltage', 'string'),
 ('Global_intensity', 'string'),
 ('Sub_metering_1', 'string'),
 ('Sub_metering_2', 'string'),
 ('Sub_metering_3', 'string')]

In [18]:
data = (data.withColumn('Date', data.Date.cast('date'))
        .withColumn('Time', data.Time.cast('timestamp'))
        .withColumn('Global_active_power', data.Global_active_power.cast('float'))
        .withColumn('Global_reactive_power', data.Global_reactive_power.cast('float'))
        .withColumn('Voltage', data.Voltage.cast('float'))
        .withColumn('Global_intensity', data.Global_intensity.cast('float'))
        .withColumn('Sub_metering_1', data.Sub_metering_1.cast('float'))
        .withColumn('Sub_metering_2', data.Sub_metering_2.cast('float'))
        .withColumn('Sub_metering_3', data.Sub_metering_3.cast('float')))
data.dtypes

[('Date', 'date'),
 ('Time', 'timestamp'),
 ('Global_active_power', 'float'),
 ('Global_reactive_power', 'float'),
 ('Voltage', 'float'),
 ('Global_intensity', 'float'),
 ('Sub_metering_1', 'float'),
 ('Sub_metering_2', 'float'),
 ('Sub_metering_3', 'float')]

In [19]:
data = data.drop('Date', 'Time', 'Sub_metering_1', 
        'Sub_metering_2', 
        'Sub_metering_3')

1. Output the **minimum**, **maximum**, and **count** of the columns:`Global_active_power`, `Global_reactive_power`, `Voltage`, and `Global_intensity`
2. Output the **mean** and **standard deviation** of these columns


In [20]:
features = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']

In [21]:
summary = data.select(features).describe()
summary.show()

+-------+-------------------+---------------------+------------------+-----------------+
|summary|Global_active_power|Global_reactive_power|           Voltage| Global_intensity|
+-------+-------------------+---------------------+------------------+-----------------+
|  count|            2049280|              2049280|           2049280|          2049280|
|   mean| 1.0916150366540094|   0.1237144765251571|240.83985796672414|4.627759313004169|
| stddev| 1.0572941611179578|  0.11272197958641265| 3.239986661206364|4.444396258981228|
|    min|              0.076|                  0.0|             223.2|              0.2|
|    max|             11.122|                 1.39|            254.15|             48.4|
+-------+-------------------+---------------------+------------------+-----------------+



In [22]:
minmax = summary.toPandas().set_index('summary').loc[['min', 'max']].apply(pd.to_numeric)

3 . Perform **min-max normalization** on the columns to generate normalized output

In [23]:
for feature in features:
    min = minmax[feature].loc['min']
    max = minmax[feature].loc['max'] 
    (data.withColumn('norm_' + feature, (data[feature]-min)/(max-min))
         .select(feature, 'norm_' + feature)
         .show())

+-------------------+------------------------+
|Global_active_power|norm_Global_active_power|
+-------------------+------------------------+
|              4.216|      0.3747963136075179|
|               5.36|      0.4783632204883582|
|              5.374|      0.4796306420857549|
|              5.388|      0.4808980636831516|
|              3.666|     0.32500451651039103|
|               3.52|      0.3117870705166136|
|              3.702|      0.3282636160352363|
|                3.7|     0.32808256814083975|
|              3.668|      0.3251855859889428|
|              3.662|     0.32464239913744275|
|              4.448|      0.3957993802483825|
|              5.412|     0.48307081108915195|
|              5.224|      0.4660510571348739|
|              5.268|       0.470034412989771|
|              4.054|      0.3601303549536367|
|              3.384|     0.29947492874728454|
|               3.27|     0.28915444332124873|
|               3.43|     0.30363933249657815|
|            

In [24]:
sc.stop()